# GAN

## 例題
MNISTデータセット（手書き数字文字画像）をGANで生成するプログラムを作成せよ．

### STEP1：インポート
各種ライブラリをインポートする．


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

### STEP2：ネットワーク定義
ネットワークを定義するクラスを記述する． 

#### Discriminatorの定義

- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
- 全結合層は```nn.Linear```クラスを用いる
- 活性化関数にはLeakyReLU（```F.leaky_relu()```）を用いる
- 最終段の活性化関数にはsigmoid関数（`torch.sigmoid()`）を追加して，出力を0から1に収める


In [ ]:
# 任意のクラス名（ここではDiscriminator）をつける
class Discriminator(nn.Module):
    def _init_weights(self): # 重みの初期化
        for weight in self.parameters():
            nn.init.normal_(weight, 0.0, 0.02)

    def __init__(self, in_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 384, bias=False)
        self.fc2 = nn.Linear(384, 128, bias=False)
        self.fc3 = nn.Linear(128, 32, bias=False)
        self.fc4 = nn.Linear(32, 1, bias=False)
        self._init_weights()

    def forward(self, x):
        x = self.fc1(x)
        x = F.leaky_relu(x)

        x = self.fc2(x)
        x = F.leaky_relu(x)

        x = self.fc3(x)
        x = F.leaky_relu(x)

        x = self.fc4(x)
        x = torch.sigmoid(x)  # 0 to 1 (0: fake, 1: true)

        return x

In [ ]:
# 上記Discriminatorクラスのインスタンスを生成
in_features = 1 * 28 * 28
dis = Discriminator(in_features)

# 使えるデバイスを探索
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 実行するデバイスにインスタンスを移動
dis.to(device)

#### Generatorの定義
- クラスはnn.Moduleを継承する
- クラスは```__init__()```と```forward()```というメソッドを定義する
    - ```__init__()```：ネットワークの構造を定義
    - ```forward()```：ネットワークの順伝播を定義
- 全結合層は```nn.Linear```クラスを用いる
- 活性化関数にはReLU（```F.relu()```）を用いる
- 最終段の活性化関数にはtanh関数（`torch.tanh()`）を追加して，出力を-1から1に収める

In [ ]:
class Generator(nn.Module):
    def _init_weights(self): # 重みの初期化
        for weight in self.parameters():
            nn.init.normal_(weight, 0.0, 0.02)

    def __init__(self, zsize, in_features):
        super().__init__()
        self.fc1 = nn.Linear(zsize, 256, bias=False)
        self.fc2 = nn.Linear(256, 512, bias=False)
        self.fc3 = nn.Linear(512, 1024, bias=False)
        self.fc4 = nn.Linear(1024, in_features, bias=False)
        self._init_weights()
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.fc3(x)
        x = F.relu(x)

        x = self.fc4(x)
        x = torch.tanh(x)  # -1 to 1

        return x

In [ ]:
# 上記Generatorクラスのインスタンスを生成
zsize = 100
gen = Generator(zsize, in_features)

# 実行するデバイスにインスタンスを移動
gen.to(device)

### STEP3：誤差関数・最適化器
誤差関数と最適化器を定義する．
- 誤差関数：今回は`nn.BCELoss`（Binary Cross Entropy Loss）を使う
    - GANでよく使われる
- 最適化器：```optim.SGD```と```optim.Adam```がよく使われる
    - ```optim.SGD```：確率的勾配降下法
    - ```optim.Adam```：Adam（今回はこちらを使う）

In [ ]:
# 誤差関数（バイナリクロスエントロピー誤差）の定義
criterion = nn.BCELoss()

# 最適化器（Adam）の定義
# 引数1：最適化するネットワークのパラメータ parameters()で取得可能
# 引数2：学習率 = 0.0002
# 引数3：ハイパーパラメータ（ベータ）
optimizer_dis = optim.Adam(dis.parameters(), lr=0.0002, betas=[0.5, 0.999])
optimizer_gen = optim.Adam(gen.parameters(), lr=0.0002, betas=[0.5, 0.999])

### STEP4：データセット
データセットを用意する．

- MNISTデータセット（28x28ピクセルの画像）は```torchvision```から読み出す
- バッチサイズ（一度にネットワークに入力するデータ数）を指定する
    - 大→データセット全てを入力するまでにかかる繰り返し回数が減るため学習時間が短くなるが メモリの消費が大きくなる
    - 小→データセット全てを入力するまでにかかる繰り返し回数が増えるため学習時間が長くなるが メモリの消費が小さくなる

In [ ]:
# ローカル環境で動作させる場合は
#   torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# でデータセットを./dataフォルダにダウンロードできるが，Colabだとダウンロードエラーが生じることがある． 
# 代わりにデータセットを直接ダウンロードし，ダウンロードした場所を指定する対策をとっている．

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [ ]:
# 画像整形の設定
transform_mnist = transforms.Compose([
    # PyTorchではTensor型と呼ばれる配列でデータを取り扱う
    # ここではダウンロードしてきた画像をTensor型に変換する
    transforms.ToTensor(),

    # ダウンロードした画像の値域は[0, 1]だが これを[-1, 1]に整形する
    # 入力画像inputsは下記の式に基づいて変換される
    # inputs[channel] = (inputs[channel] - mean[channel]) / std[channel]
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ])

# データセットの読み込み
# 引数1：データセットのパス
# 引数2：Train用かTest用か
# 引数3：ダウンロードしてくるか否か
# 引数4：画像整形の設定
trainset_mnist = torchvision.datasets.MNIST(root='./', train=True, download=False, transform=transform_mnist)

# ミニバッチの設定
# 引数1：データセット
# 引数2：バッチサイズ
# 引数3：データをシャッフルするか否か
batchsize = 100
trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, batch_size=batchsize, shuffle=True)

### STEP5：ネットワーク学習
学習を下記のステップで行う．
- まず，識別器の学習を行う
    - 今回はリアル画像のラベルが1，フェイク画像のラベルが0である
    1. MNISTの画像（学習対象な本物の画像）を入力する
    2. フェイク画像（Generatorで生成した画像）を入力する
    - それぞれ順伝播，誤差の計算，逆伝播をして識別器の性能を上げる
- 次に，生成器の学習を行う
- 以上を繰り返す（適宜ネットワークを評価）

In [ ]:
# 画像表示関数
def imshow(img):
    img = torchvision.utils.make_grid(img)
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.figure(figsize=(12, 12))
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.show()

In [ ]:
one_labels = torch.ones(batchsize).to(device)
zero_labels = torch.zeros(batchsize).to(device)

losses_dis = []
losses_gen = []

# エポック数：データセットを何周するか
EPOCHS = 50

for epoch in range(EPOCHS):
    running_loss_dis = 0.0
    running_loss_gen = 0.0

    # for文でTraining用データセットをループする
    for i, data in enumerate(trainloader_mnist, 1):

        # inputs：入力データ
        # labels：教師データ
        inputs, labels = data

        # 実行するデバイスにデータを移動
        inputs = inputs.to(device)
        labels = labels.to(device)

        #================
        # 識別器の学習
        #================
        #----------------
        # リアル画像（MNISTの画像）を入力する
        #----------------

        # ネットワークパラメータの微分値をリセット
        dis.zero_grad()

        # データの形状を変換
        real_imgs = inputs.reshape(batchsize, -1)

        # 入力データを順伝播し出力を確認
        output_real_imgs = dis(real_imgs)

        # データの形状を変換
        output_real_imgs = output_real_imgs.reshape(batchsize)

        # ネットワークの出力と，正しいラベル（全てリアル画像なので1）との
        # 誤差を計算
        loss_real_imgs = criterion(output_real_imgs, one_labels)
        loss_real_imgs.backward() # 誤差逆伝播

        #----------------
        # フェイク画像（Generatorで生成した画像）を入力する
        #----------------

        # 乱数画像を生成
        z = torch.randn(batchsize, zsize).to(device)

        # 乱数を入力に，Generatorネットワークで画像を生成
        fake_imgs = gen(z)

        # Generatorネットワークで生成した画像を識別機へ入力
        output_fake_imgs = dis(fake_imgs)

        # データの形状を変換
        output_fake_imgs = output_fake_imgs.reshape(batchsize)

        # ネットワークの出力と，正しいラベル（全てフェイク画像なので0）との
        # 誤差を計算
        loss_fake_imgs = criterion(output_fake_imgs, zero_labels)
        loss_fake_imgs.backward() # 誤差逆伝播

        #----------------
        # 最終的な損失を計算する
        #----------------
        loss_dis = loss_real_imgs + loss_fake_imgs
        optimizer_dis.step()
        running_loss_dis += loss_dis

        #================
        # 生成器の学習
        #================

        # ネットワークパラメータの微分値をリセット
        gen.zero_grad()
        
        # 乱数画像を生成
        z = torch.randn(batchsize, zsize).to(device)

        # 乱数を入力に，Generatorネットワークで画像を生成
        fake_imgs = gen(z)

        # Generatorネットワークで生成した画像を識別機へ入力
        output_fake_imgs = dis(fake_imgs)

        # データの形状を変換
        output_fake_imgs = output_fake_imgs.reshape(batchsize)

        # 誤差計算
        loss_gen = criterion(output_fake_imgs, one_labels)
        loss_gen.backward() # 誤差逆伝播
        optimizer_gen.step()

        # 損失の計算
        running_loss_gen += loss_gen


    running_loss_dis /= i
    running_loss_gen /= i
    print(f'epoch: {epoch}, dis loss: {running_loss_dis}, gen loss: {running_loss_gen}')
    losses_dis.append(running_loss_dis.cpu())
    losses_gen.append(running_loss_gen.cpu())
    if (epoch+1) % 10 == 0:
        z = torch.randn(batchsize, zsize).to(device)
        generated_imgs = gen(z).cpu()
        imshow(generated_imgs[0:8].reshape(8, 1, 28, 28))

## 演習
MNISTデータセット（手書き数字文字画像）をDCGANで生成するプログラムを作成せよ．  
DCGAN：Deep Convolutional GANの略であり，畳み込み層で構成されたGANである．

### STEP1：インポート

### STEP2：ネットワーク定義

#### Discriminatorの定義
Note:
構造は，一例として

- 1-1. 畳み込み層
- 1-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 1-3. 活性化関数（LeakyRelu）
- 2-1. 畳み込み層
- 2-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 2-3. 活性化関数（LeakyRelu）
- 3-1. 畳み込み層
- 3-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 3-3. 活性化関数（LeakyRelu）
- 4-1. 畳み込み層
- 4-3. 活性化関数（sigmoid）

等が考えられる

#### Generatorの定義
Note:
構造は，一例として

- 1-1. 特殊な畳み込み層（ConvTranspose2d）: `nn.ConvTranspose2d()`
- 1-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 1-3. 活性化関数（Relu）
- 2-1. 特殊な畳み込み層（ConvTranspose2d）
- 2-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 2-3. 活性化関数（Relu）
- 3-1. 特殊な畳み込み層（ConvTranspose2d）
- 3-2. バッチノーマライゼーション層: `nn.BatchNorm2d()`
- 3-3. 活性化関数（Relu）
- 4-1. 特殊な畳み込み層（ConvTranspose2d）
- 4-3. 活性化関数（ハイパボリックtan）: `F.tanh()`

等が考えられる

`ConvTranspose2d`は特殊な畳み込み処理を行う層であり，入力よりも出力の方が大きな特徴マップを持つ

### STEP3：誤差関数・最適化器

### STEP4：データセット


In [ ]:
# ローカル環境で動作させる場合は
#   torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# でデータセットを./dataフォルダにダウンロードできるが，Colabだとダウンロードエラーが生じることがある． 
# 代わりにデータセットを直接ダウンロードし，ダウンロードした場所を指定する対策をとっている．

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

In [ ]:
# 画像整形の設定
transform_mnist = transforms.Compose([
    # PyTorchではTensor型と呼ばれる配列でデータを取り扱う
    # ここではダウンロードしてきた画像をTensor型に変換する
    transforms.ToTensor(),

    # ダウンロードした画像の値域は[0, 1]だが これを[-1, 1]に整形する
    # 入力画像inputsは下記の式に基づいて変換される
    # inputs[channel] = (inputs[channel] - mean[channel]) / std[channel]
    transforms.Normalize(mean=(0.5, ), std=(0.5, ))
    ])

# データセットの読み込み
# 引数1：データセットのパス
# 引数2：Training用かTest用か
# 引数3：ダウンロードしてくるか否か
# 引数4：画像整形の設定
trainset_mnist = torchvision.datasets.MNIST(root='./', train=True, download=False, transform=transform_mnist)

# ミニバッチの設定
# 引数1：データセット
# 引数2：バッチサイズ
# 引数3：データをシャッフルするか否か
# Trainingはデータの順番に依存するため 偏りが生じないようにシャッフルすることがある
# Testはデータの順番に依存しないため シャッフルは不要
batchsize = 100
trainloader_mnist = torch.utils.data.DataLoader(trainset_mnist, batch_size=batchsize, shuffle=True)

### STEP5：ネットワーク学習